In [1]:
import os
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from data.dataset import TextResponseDataset
import pandas as pd
import os
import itertools as it

In [53]:
metrics = {'mse':0, 'auc':1, 'll':2, 'acc':3, 'perp':4, 'npmi':5, 'shuffle':6}
method_names = {'hstm-all':'HSTM', 'bert':'BERT', 'stm':'STM', 'stm+bow':'HSTM-Het', 'slda':'sLDA'}
out = os.path.join('..', 'out')
lda_pretrained = {'hstm-all', 'stm', 'stm+bow'}

models = ['hstm-all', 'stm', 'stm+bow', 'slda', 'bert']

datasets = ['amazon', 
            'amazon_binary', 
            'peerread', 
            'yelp', 
            'immigration', 
            'samesex', 
            'deathpenalty', 
            'guncontrol']

framing_corpus_topics = {'immigration', 'samesex', 'deathpenalty', 'guncontrol'}

Cs = [1e-4, 5e-5, 1e-5, 5e-6, 1e-6]
Ctopics = [1e-4, 5e-5, 1e-5, 5e-6, 1e-6]
num_folds = 5
n_metrics = len(metrics.keys())

In [78]:
def get_filename(model, dataset, e_idx, is_mfc=False, C=None, Ct=None):
    base_dir = dataset
    if is_mfc and model != 'bert':
        base_dir = 'framing_corpus_' + dataset
    if model in lda_pretrained:
        base_dir += '.lda_pretrained'
    
    fname = model + '.result.split'+str(e_idx)

    if C is not None and Ct is not None:
        fname += '.setting=' + str((C,Ct))
    elif C is not None:
        fname += '.setting=' + str((C,1e-6))
        
    return os.path.join(out, base_dir, fname)
        
def create_results_table(exp_results, model_settings_dict, dataset):
    data = []
    for model in models:
        if dataset in TextResponseDataset.CLASSIFICATION_SETTINGS or dataset in framing_corpus_topics:
            metric_idx = metrics['acc']
        else:
            metric_idx = metrics['mse']
            
        values = [method_names[model]]
        setting = model_settings_dict[model][dataset]        
        average_error = exp_results[dataset][model][setting].mean(axis=0)[metric_idx]
        std = exp_results[dataset][model][setting].std(axis=0)[metric_idx]
        values += [str(np.round(average_error, 2)) + ' (' + str(np.round(std,4)) + ')']
        data.append(values)
    results_df = pd.DataFrame(data, columns=['Model name', 'Mean performance'])
    return results_df

def get_best_results(exp_results, data, model, is_classification=False):
    best_score = 1e7 if not is_classification else 0.
    best_config = None
    metric_idx = metrics['mse'] if not is_classification else metrics['acc']
    
    for config, res in exp_results[data][model].items():
        mean_results = res.mean(axis=0)
        current_score = mean_results[metric_idx]
        if not is_classification:
            if current_score < best_score:
                best_score = current_score
                best_config = config
        else:
            if current_score > best_score:
                best_score = current_score
                best_config = config
    return best_config

In [79]:
exp_results = {data:{'hstm-all':{(C,Ct):np.zeros((num_folds,n_metrics)) for C in Cs for Ct in Ctopics}}
                     for data in datasets}
for data in datasets:
    exp_results[data].update({'stm+bow':{(C,1e-6):np.zeros((num_folds,n_metrics)) for C in Cs}}) 
    exp_results[data].update({'stm':{(C,1e-6):np.zeros((num_folds,n_metrics)) for C in Cs}}) 
    exp_results[data].update({'bert':{(0.,0):np.zeros((num_folds,n_metrics-3))}})
    exp_results[data].update({'slda':{(0.,0):np.zeros((num_folds,n_metrics-1))}})

In [80]:
for data in ['deathpenalty']:
    is_framing_corpus = False
    if data in framing_corpus_topics:
        is_framing_corpus=True
    
    for model in models:
        for e_idx in range(num_folds):
            if model == 'bert' or model == 'slda':
                filename = get_filename(model, data, e_idx, is_mfc=is_framing_corpus)
                results = np.load(filename + '.npy')
                exp_results[data][model][(0.,0.,)][e_idx]=results
            elif model == 'stm' or model == 'stm+bow':
                for C in Cs:
                    filename = get_filename(model, data, e_idx, is_mfc=is_framing_corpus, C=C)
                    results = np.load(filename + '.npy')
                    exp_results[data][model][(C,1e-6)][e_idx]=results
            else:
                for (C, Ct) in it.product(Cs, Ctopics):
                    filename = get_filename(model, data, e_idx, is_mfc=is_framing_corpus, C=C, Ct=Ct)
                    results = np.load(filename + '.npy')
                    exp_results[data][model][(C,Ct)][e_idx]=results

In [81]:
model_settings_dict = {model:{data:None for data in datasets} for model in models}
for model in models:
    for data in ['deathpenalty']:
        if model == 'bert' or model == 'slda':
            model_settings_dict[model][data] = (0.,0.)
        else:
            is_classification = (data in TextResponseDataset.CLASSIFICATION_SETTINGS) or (data in framing_corpus_topics)
            model_settings_dict[model][data] = get_best_results(exp_results, 
                                                                data, 
                                                                model, 
                                                                is_classification=is_classification)

In [82]:
df = create_results_table(exp_results, model_settings_dict, 'deathpenalty')
df

,Model name,Mean performance
0,HSTM,0.78 (0.0233)
1,STM,0.69 (0.0139)
2,HSTM-Het,0.76 (0.0138)
3,sLDA,0.69 (0.0139)
4,BERT,0.7 (0.016)
